### Generate Candlestick chart using historical OHLC data

In [109]:
import plotly.graph_objects as go
import plotly.io as pio
import argparse
import pandas as pd

pio.renderers.default = 'browser'
display_data = 2000

ohlc_df = pd.read_csv(f'../data/btc.csv', nrows=display_data)
fig = go.Figure(
    data=[go.Candlestick(
        x=ohlc_df['Timestamp'], 
        open=ohlc_df['Open'], 
        high=ohlc_df['High'], 
        low=ohlc_df['Low'], 
        close=ohlc_df['Close'],
        increasing_line_color= '#29d691',
        decreasing_line_color= '#ee113e'
    )], 
)

### Plot all closed positions using gray ⚪ for entry time and either 🟢 if PnL > 0 or 🔴 otherwise

In [112]:
positions_df = pd.read_csv('../results/closed_positions.csv', nrows=display_data // 2)
last_ohlc_timestamp = ohlc_df['Timestamp'].iloc[-1]
position_id = 1

for index, row in positions_df.iterrows():
    if row['Exit'] > last_ohlc_timestamp:
        continue

    color = 'green' if int(row['PnL']) > 0 else 'red'
    entry_low = ohlc_df.loc[ohlc_df['Timestamp'] == row['Entry'], 'Low'].values[0]
    exit_low = ohlc_df.loc[ohlc_df['Timestamp'] == row['Exit'], 'Low'].values[0]

    fig.add_trace(go.Scatter(
        x=[row['Entry']],
        y=[entry_low],
        mode='markers',
        marker=dict(
            size=22, 
            color='rgba(131, 131, 131, 0.7)', 
            symbol='circle'
        ),
        name='Entry',
        text=[f"Entry: {row['Entry']}<br>Position ID: {position_id}<br>Type: {row['Type'].capitalize()}"],
        hoverinfo='text'
    ))

    fig.add_trace(go.Scatter(
        x=[row['Exit']],
        y=[exit_low],
        mode='markers',
        marker=dict(
            size=22,
            symbol='circle',
            color=['rgba(0, 200, 0, 0.4)' if row['PnL'] > 0 else 'rgba(255, 0, 0, 0.4)']
        ),
        name='Exit',
        text=[f"Exit: {row['Exit']}<br>Position ID: {position_id}<br>Type: {row['Type'].capitalize()}<br>PnL: {row['PnL']}"],
        hoverinfo='text'
    ))

    position_id += 1

### Plot all trades using 🟢▲ for Buy trades and 🔴▼ for Sell trades

In [113]:
trades_df = pd.read_csv('../results/trades.csv', nrows=display_data)

for index, row, in trades_df.iterrows():
    if row['Timestamp'] > last_ohlc_timestamp:
        continue

    color = 'green' if row['Side'] == 'Buy' else 'red'
    fig.add_trace(go.Scatter(
        x=[row['Timestamp']],
        y=[row['Price']],
        mode='markers',
        name=row['Side'],
        marker=dict(
            symbol=('triangle-down' if row['Side'] == 'Buy' else 'triangle-up'), 
            size=10, 
            color=color, 
            line=dict(width=1, color='black')
        ),
        hovertemplate=f"{row['Side']}<br>Price: ${row['Price']}<br>Quantity: {row['Quantity']}<br>Time: {row['Timestamp']}",
    ))

fig.update_layout(template="ggplot2", title=dict(text="Trading Strategy Performance Visualisation"))
fig.show()